In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time

2024-11-16 18:08:18.153382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731809298.172179   85761 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731809298.177964   85761 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 18:08:18.213516: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [119]:
torch.cuda.empty_cache()


In [5]:
#Image Preporcessing 

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(40),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(40),
    transforms.CenterCrop(40),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
# Dataset - Replace with paths to your own dataset
train_data = datasets.ImageFolder(root='/home/harshal/ml/dataset/train', transform=train_transforms)
test_data = datasets.ImageFolder(root='/home/harshal/ml/dataset/test', transform=test_transforms)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

In [7]:
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    return correct / total

In [8]:
def train_model3(model, train_loader, test_loader, num_epochs=10, device='cuda', use_tensorboard=False):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    model_name = model.__class__.__name__
    
    if use_tensorboard:
        writer = SummaryWriter(f'runs/experiment/{model_name}')

    model.to(device)  # Move model to the selected device (GPU or CPU)
    global_step = 0  # Keep track of the global step for iterations
    
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        start_time = time.time()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to device

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

            # Log metrics after each iteration
            if use_tensorboard:
                # Log training loss per iteration
                writer.add_scalar('Training Loss', loss.item(), global_step)
                
                # Log training accuracy per iteration
                train_accuracy = correct / total
                writer.add_scalar('Training Accuracy', train_accuracy, global_step)

                # Log testing accuracy per iteration
                test_accuracy = calculate_accuracy(model, test_loader, device)
                writer.add_scalar('Testing Accuracy', test_accuracy, global_step)

            global_step += 1  

        epoch_time = time.time() - start_time

        # Initialize lists outside the if block to avoid UnboundLocalError
        all_test_images = []
        all_preds = []  

        if use_tensorboard:
            model.eval()  
            with torch.no_grad():  
                for inputs, _ in test_loader:
                    inputs = inputs.to(device)

                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    # Append images and predictions
                    all_test_images.append(inputs.cpu())
                    all_preds.append(preds.cpu())

            # Concatenate all test images and predictions
            all_test_images = torch.cat(all_test_images, dim=0)
            all_preds = torch.cat(all_preds, dim=0)

            # Log test images and their predictions to TensorBoard
            img_grid = torchvision.utils.make_grid(all_test_images[:74])  # Log all 74 images
            writer.add_image('Test Images', img_grid)

            # Log predictions as text
            predicted_labels = ', '.join(map(str, all_preds.numpy()))  # Log all predicted labels
            writer.add_text('Predicted Labels', predicted_labels)

        # Log the training loss and accuracy at the end of each epoch
        train_accuracy = correct / total
        test_accuracy = calculate_accuracy(model, test_loader, device)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}, Time: {epoch_time:.2f}s")

    if use_tensorboard:
        writer.close()  # Close the writer after training completes
    torch.save(model.state_dict(), f'{model_name}.pth')
    print(f"Model saved to '{model_name}.pth'")


    return model


In [69]:
def train_model(model, train_loader, test_loader, num_epochs=10, device='cuda', use_tensorboard=False):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    if use_tensorboard:
        writer = SummaryWriter('runs/experiment')

    model.to(device)  # Move model to the selected device (GPU or CPU)
    global_step = 0  # Keep track of the global step for iterations
    
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        start_time = time.time()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to device

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

            # Log metrics after each iteration
            if use_tensorboard:
                # Log training loss per iteration
                writer.add_scalar('Training Loss', loss.item(), global_step)
                
                # Log training accuracy per iteration
                train_accuracy = correct / total
                writer.add_scalar('Training Accuracy', train_accuracy, global_step)

                # Log testing accuracy per iteration
                test_accuracy = calculate_accuracy(model, test_loader, device)
                writer.add_scalar('Testing Accuracy', test_accuracy, global_step)

            global_step += 1  

        epoch_time = time.time() - start_time

        if use_tensorboard:
            model.eval()  # Set model to evaluation mode
            all_test_images = []
            all_preds = []  

            with torch.no_grad():  # No need to compute gradients for test data
                for inputs, _ in test_loader:
                    inputs = inputs.to(device)

                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    # Append images and predictions
                    all_test_images.append(inputs.cpu())
                    all_preds.append(preds.cpu())

        # Concatenate all test images and predictions
        all_test_images = torch.cat(all_test_images, dim=0)
        all_preds = torch.cat(all_preds, dim=0)

        # Log test images and their predictions to TensorBoard
        img_grid = torchvision.utils.make_grid(all_test_images[:74])  # Log all 74 images
        writer.add_image('Test Images', img_grid)

        # Log predictions as text
        predicted_labels = ', '.join(map(str, all_preds.numpy()))  # Log all predicted labels
        writer.add_text('Predicted Labels', predicted_labels)

        # Log the training loss and accuracy at the end of each epoch
        train_accuracy = correct / total
        test_accuracy = calculate_accuracy(model, test_loader, device)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}, Time: {epoch_time:.2f}s")


    if use_tensorboard:
        writer.close()  # Close the writer after training completes

    return model

In [ ]:
class VGG1Block(nn.Module):
    def __init__(self):
        super(VGG1Block, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)        
        )
        self.fc = nn.Sequential(
            nn.Linear(32 * 20 * 20, 512),               
            nn.ReLU(),
            nn.Linear(512, 2)                           
        )
    
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1) 
        x = self.fc(x)
        return x


In [124]:
torch.cuda.empty_cache()


In [106]:
vgg1 = VGG1Block().to(device)
trained_model = train_model3(vgg1, train_loader, test_loader, num_epochs=20, device=device, use_tensorboard=True)


Epoch [1/20], Loss: 0.8238, Train Accuracy: 0.5538, Test Accuracy: 0.6267, Time: 1.22s
Epoch [2/20], Loss: 0.6522, Train Accuracy: 0.5949, Test Accuracy: 0.7200, Time: 0.84s
Epoch [3/20], Loss: 0.6085, Train Accuracy: 0.6667, Test Accuracy: 0.7333, Time: 0.85s
Epoch [4/20], Loss: 0.5970, Train Accuracy: 0.7077, Test Accuracy: 0.7467, Time: 0.85s
Epoch [5/20], Loss: 0.5560, Train Accuracy: 0.7282, Test Accuracy: 0.8267, Time: 0.83s
Epoch [6/20], Loss: 0.5704, Train Accuracy: 0.7385, Test Accuracy: 0.8267, Time: 0.82s
Epoch [7/20], Loss: 0.5780, Train Accuracy: 0.7231, Test Accuracy: 0.7867, Time: 0.81s
Epoch [8/20], Loss: 0.5183, Train Accuracy: 0.7590, Test Accuracy: 0.7867, Time: 0.82s
Epoch [9/20], Loss: 0.5454, Train Accuracy: 0.7179, Test Accuracy: 0.8133, Time: 0.82s
Epoch [10/20], Loss: 0.5033, Train Accuracy: 0.7487, Test Accuracy: 0.8267, Time: 0.81s
Epoch [11/20], Loss: 0.4920, Train Accuracy: 0.7846, Test Accuracy: 0.8000, Time: 0.82s
Epoch [12/20], Loss: 0.4582, Train Accura

In [55]:
class VGG3Block(nn.Module):
    def __init__(self):
        super(VGG3Block, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Conv1
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # MaxPool1
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # Conv2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # MaxPool2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Conv3
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPool3
        )
        # The input size of the first Linear layer should be 128*5*5 = 3200
        self.fc = nn.Sequential(
            nn.Linear(128 * 5 * 5, 512),  # Adjusted for correct input size
            nn.ReLU(),
            nn.Linear(512, 2)  # Output layer for 2 classes
        )
    
    def forward(self, x):
        x = self.features(x)  # Pass through convolutional layers
        x = torch.flatten(x, 1)  # Flatten the output
        x = self.fc(x)  # Pass through fully connected layers
        return x

In [110]:
vgg3 = VGG3Block().to(device)
trained_model = train_model3(vgg3, train_loader, test_loader, num_epochs=20, device=device, use_tensorboard=True)

Epoch [1/20], Loss: 0.6744, Train Accuracy: 0.6154, Test Accuracy: 0.6400, Time: 1.12s
Epoch [2/20], Loss: 0.6265, Train Accuracy: 0.6718, Test Accuracy: 0.7733, Time: 0.90s
Epoch [3/20], Loss: 0.5712, Train Accuracy: 0.7538, Test Accuracy: 0.7733, Time: 0.69s
Epoch [4/20], Loss: 0.5689, Train Accuracy: 0.7231, Test Accuracy: 0.7600, Time: 0.59s
Epoch [5/20], Loss: 0.5205, Train Accuracy: 0.7282, Test Accuracy: 0.7200, Time: 0.59s
Epoch [6/20], Loss: 0.5810, Train Accuracy: 0.7179, Test Accuracy: 0.7733, Time: 0.66s
Epoch [7/20], Loss: 0.6243, Train Accuracy: 0.6872, Test Accuracy: 0.7467, Time: 0.68s
Epoch [8/20], Loss: 0.5814, Train Accuracy: 0.7128, Test Accuracy: 0.8133, Time: 0.71s
Epoch [9/20], Loss: 0.5009, Train Accuracy: 0.7538, Test Accuracy: 0.8000, Time: 0.72s
Epoch [10/20], Loss: 0.5475, Train Accuracy: 0.7333, Test Accuracy: 0.7600, Time: 0.86s
Epoch [11/20], Loss: 0.5229, Train Accuracy: 0.7333, Test Accuracy: 0.7867, Time: 0.93s
Epoch [12/20], Loss: 0.5126, Train Accura

In [111]:
#Data_augumentation 
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(40),  # Randomly crop to 40x40
        transforms.RandomHorizontalFlip(),  # Random horizontal flip
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    ]),
    'val': transforms.Compose([
        transforms.Resize(40),  # Resize to 40x40
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

# Load datasets with augmentations
train_dataset = datasets.ImageFolder('/home/harshal/ml/dataset/train', transform=data_transforms['train'])
val_dataset = datasets.ImageFolder('/home/harshal/ml/dataset/test', transform=data_transforms['val'])

train_loader_transformed = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader_transformed = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [75]:
def train_model2(model, train_loader, test_loader, num_epochs=10, device='cuda', use_tensorboard=False):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    if use_tensorboard:
        writer = SummaryWriter('runs/experiment')

    model.to(device)  # Move model to the selected device (GPU or CPU)
    global_step = 0  # Keep track of the global step for iterations
    
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        start_time = time.time()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to device

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

            # Log metrics after each iteration
            if use_tensorboard:
                # Log training loss per iteration
                writer.add_scalar('Training Loss', loss.item(), global_step)
                
                # Log training accuracy per iteration
                train_accuracy = correct / total
                writer.add_scalar('Training Accuracy', train_accuracy, global_step)

                # Log testing accuracy per iteration
                test_accuracy = calculate_accuracy(model, test_loader, device)
                writer.add_scalar('Testing Accuracy', test_accuracy, global_step)

            global_step += 1  

        epoch_time = time.time() - start_time

        # Initialize lists outside the if block to avoid UnboundLocalError
        all_test_images = []
        all_preds = []  

        if use_tensorboard:
            model.eval()  # Set model to evaluation mode
            with torch.no_grad():  # No need to compute gradients for test data
                for inputs, _ in test_loader:
                    inputs = inputs.to(device)

                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    # Append images and predictions
                    all_test_images.append(inputs.cpu())
                    all_preds.append(preds.cpu())

            # Concatenate all test images and predictions
            all_test_images = torch.cat(all_test_images, dim=0)
            all_preds = torch.cat(all_preds, dim=0)

            # Log test images and their predictions to TensorBoard
            img_grid = torchvision.utils.make_grid(all_test_images[:74])  # Log all 74 images
            writer.add_image('Test Images', img_grid)

            # Log predictions as text
            predicted_labels = ', '.join(map(str, all_preds.numpy()))  # Log all predicted labels
            writer.add_text('Predicted Labels', predicted_labels)

        # Log the training loss and accuracy at the end of each epoch
        train_accuracy = correct / total
        test_accuracy = calculate_accuracy(model, test_loader, device)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}, Time: {epoch_time:.2f}s")

    if use_tensorboard:
        writer.close()  # Close the writer after training completes

    return model


In [112]:
vgg3_with_agumentation = VGG3Block().to(device)
trained_model = train_model3(vgg3_with_agumentation, train_loader_transformed, test_loader_transformed, num_epochs=20, device=device,use_tensorboard=True)

Epoch [1/20], Loss: 0.6751, Train Accuracy: 0.5641, Test Accuracy: 0.4933, Time: 1.18s
Epoch [2/20], Loss: 0.6869, Train Accuracy: 0.5436, Test Accuracy: 0.7200, Time: 0.94s
Epoch [3/20], Loss: 0.6433, Train Accuracy: 0.6462, Test Accuracy: 0.7067, Time: 0.94s
Epoch [4/20], Loss: 0.6072, Train Accuracy: 0.6769, Test Accuracy: 0.7333, Time: 0.93s
Epoch [5/20], Loss: 0.6300, Train Accuracy: 0.6256, Test Accuracy: 0.6800, Time: 0.93s
Epoch [6/20], Loss: 0.6473, Train Accuracy: 0.6256, Test Accuracy: 0.7867, Time: 0.93s
Epoch [7/20], Loss: 0.5984, Train Accuracy: 0.6974, Test Accuracy: 0.7733, Time: 0.92s
Epoch [8/20], Loss: 0.5910, Train Accuracy: 0.7128, Test Accuracy: 0.7467, Time: 0.92s
Epoch [9/20], Loss: 0.5678, Train Accuracy: 0.7128, Test Accuracy: 0.7733, Time: 0.93s
Epoch [10/20], Loss: 0.5428, Train Accuracy: 0.7179, Test Accuracy: 0.7067, Time: 0.82s
Epoch [11/20], Loss: 0.5738, Train Accuracy: 0.6821, Test Accuracy: 0.7333, Time: 0.64s
Epoch [12/20], Loss: 0.5883, Train Accura

In [9]:
class VGG16_Finetuned_AllLayers(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG16_Finetuned_AllLayers, self).__init__()
        self.model = models.vgg16(pretrained=True)  # or models.vgg19(pretrained=True)
        
        # Modify the classifier part of VGG16 to match the number of classes
        self.model.classifier[6] = nn.Linear(self.model.classifier[6].in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

In [11]:
vgg16_Finetuned_AllLayers = VGG16_Finetuned_AllLayers(num_classes=2).to(device)
trained_model = train_model3(vgg16_Finetuned_AllLayers, train_loader, test_loader, num_epochs=20, device=device,use_tensorboard=True)

/home/harshal/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/harshal/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/20], Loss: 0.5996, Train Accuracy: 0.7846, Test Accuracy: 0.8400, Time: 2.41s
Epoch [2/20], Loss: 0.5588, Train Accuracy: 0.7538, Test Accuracy: 0.8400, Time: 2.02s
Epoch [3/20], Loss: 0.3715, Train Accuracy: 0.8308, Test Accuracy: 0.8400, Time: 2.15s
Epoch [4/20], Loss: 0.3453, Train Accuracy: 0.8667, Test Accuracy: 0.9067, Time: 2.13s
Epoch [5/20], Loss: 0.4551, Train Accuracy: 0.8615, Test Accuracy: 0.9333, Time: 2.01s
Epoch [6/20], Loss: 0.2651, Train Accuracy: 0.8974, Test Accuracy: 0.9333, Time: 2.02s
Epoch [7/20], Loss: 0.2490, Train Accuracy: 0.9282, Test Accuracy: 0.9867, Time: 2.00s
Epoch [8/20], Loss: 0.3342, Train Accuracy: 0.8667, Test Accuracy: 0.8933, Time: 2.02s
Epoch [9/20], Loss: 0.2060, Train Accuracy: 0.9231, Test Accuracy: 0.9200, Time: 2.03s
Epoch [10/20], Loss: 0.2132, Train Accuracy: 0.9179, Test Accuracy: 0.9467, Time: 2.02s
Epoch [11/20], Loss: 0.2553, Train Accuracy: 0.8718, Test Accuracy: 0.9733, Time: 2.00s
Epoch [12/20], Loss: 0.1824, Train Accura

In [12]:
class VGG16_onlymlp(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG16_onlymlp, self).__init__()
        
        # Load the pre-trained VGG19 model
        self.model = models.vgg16(pretrained=True)  # Using VGG19

        for param in self.model.features.parameters():
            param.requires_grad = False
        
        # Modify the classifier part of VGG19 to match the number of classes
        self.model.classifier[6] = nn.Linear(self.model.classifier[6].in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)


In [13]:
#for vgg with only mlplayer
vgg16_onlymlp = VGG16_onlymlp(num_classes=2).to(device)
trained_model = train_model3(vgg16_onlymlp, train_loader, test_loader, num_epochs=20, device=device,use_tensorboard=True)

/home/harshal/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/harshal/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/20], Loss: 0.6088, Train Accuracy: 0.7026, Test Accuracy: 0.8400, Time: 1.76s
Epoch [2/20], Loss: 0.2498, Train Accuracy: 0.8872, Test Accuracy: 0.8933, Time: 1.77s
Epoch [3/20], Loss: 0.3735, Train Accuracy: 0.8872, Test Accuracy: 0.8667, Time: 1.78s
Epoch [4/20], Loss: 0.3297, Train Accuracy: 0.8615, Test Accuracy: 0.8800, Time: 1.75s
Epoch [5/20], Loss: 0.2737, Train Accuracy: 0.8718, Test Accuracy: 0.8533, Time: 1.75s
Epoch [6/20], Loss: 0.2160, Train Accuracy: 0.9026, Test Accuracy: 0.8800, Time: 1.74s
Epoch [7/20], Loss: 0.2567, Train Accuracy: 0.9128, Test Accuracy: 0.9200, Time: 1.75s
Epoch [8/20], Loss: 0.1962, Train Accuracy: 0.9026, Test Accuracy: 0.8933, Time: 1.74s
Epoch [9/20], Loss: 0.1792, Train Accuracy: 0.9333, Test Accuracy: 0.8933, Time: 1.74s
Epoch [10/20], Loss: 0.2095, Train Accuracy: 0.9128, Test Accuracy: 0.9067, Time: 1.74s
Epoch [11/20], Loss: 0.1758, Train Accuracy: 0.9026, Test Accuracy: 0.8800, Time: 1.75s
Epoch [12/20], Loss: 0.1992, Train Accura

In [14]:
#MLP with parameters comparable to VGG16

class MLPComparableToVGG16(nn.Module):
    def __init__(self, num_classes=2):
        super(MLPComparableToVGG16, self).__init__()
        
        # Input layer size is 40x40x3 = 4800
        self.fc1 = nn.Linear(40 * 40 * 3, 4096)  
        self.fc2 = nn.Linear(4096, 4096)         
        self.fc3 = nn.Linear(4096, 2048)         
        self.fc4 = nn.Linear(2048, 1024)         
        self.fc5 = nn.Linear(1024, 512)         
        self.fc6 = nn.Linear(512, 256)           
        self.fc7 = nn.Linear(256, num_classes)   

        
    def forward(self, x):
        x = x.view(x.size(0), -1) 
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = self.fc7(x)  # Output layer
        return x

In [15]:
mlp_comparbletovgg = MLPComparableToVGG16().to(device)
trained_model = train_model3(mlp_comparbletovgg, train_loader, test_loader, num_epochs=20, device=device,use_tensorboard=True)

Epoch [1/20], Loss: 0.6127, Train Accuracy: 0.6256, Test Accuracy: 0.6667, Time: 1.07s
Epoch [2/20], Loss: 0.6178, Train Accuracy: 0.6923, Test Accuracy: 0.7467, Time: 0.88s
Epoch [3/20], Loss: 0.6270, Train Accuracy: 0.6923, Test Accuracy: 0.7867, Time: 0.87s
Epoch [4/20], Loss: 0.5205, Train Accuracy: 0.7077, Test Accuracy: 0.7200, Time: 0.86s
Epoch [5/20], Loss: 0.5870, Train Accuracy: 0.6974, Test Accuracy: 0.7333, Time: 0.87s
Epoch [6/20], Loss: 0.5159, Train Accuracy: 0.7282, Test Accuracy: 0.7200, Time: 0.86s
Epoch [7/20], Loss: 0.5846, Train Accuracy: 0.6769, Test Accuracy: 0.7333, Time: 0.86s
Epoch [8/20], Loss: 0.5617, Train Accuracy: 0.7026, Test Accuracy: 0.7600, Time: 0.87s
Epoch [9/20], Loss: 0.5866, Train Accuracy: 0.7128, Test Accuracy: 0.7467, Time: 0.87s
Epoch [10/20], Loss: 0.5200, Train Accuracy: 0.6821, Test Accuracy: 0.7867, Time: 0.86s
Epoch [11/20], Loss: 0.5635, Train Accuracy: 0.6872, Test Accuracy: 0.8267, Time: 0.86s
Epoch [12/20], Loss: 0.5601, Train Accura

We used Leonardo AI for image generating
https://leonardo.ai/

In [ ]:
# Using VGG1 model for classification 

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()  # Set the model to evaluation mode

# Load and preprocess the image
img_path = "path_to_your_image.jpg"
image = Image.open(img_path)

# Define transformations (resize, normalization)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

image = transform(image).unsqueeze(0)  # Add batch dimension

# Make prediction
with torch.no_grad():
    output = model(image)
    _, predicted_class = torch.max(output, 1)

print(f'Predicted Class: {predicted_class.item()}')
